<a href="https://colab.research.google.com/github/saffarizadeh/INSY4054/blob/main/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="http://saffarizadeh.com/Logo.png" width="300px"/>

# *INSY 4054: Emerging Technologies*

# **Trained Models and Transfer Learning**

Instructor: Dr. Kambiz Saffarizadeh

---

Source: https://tfhub.dev/google/nnlm-en-dim50/2

Read the complete tutorial: https://www.tensorflow.org/tutorials/keras/text_classification_with_hub

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow_hub as hub
import tensorflow_datasets as tfds

We first use `tensorflow_datasets` to load the imdb_reviews dataset.
`tensorflow_datasets` loads large datasets in a specific way that works well with TensorFlow models. The data loaded using this method is not in numpy array format. Instead it is a TensorFlow Dataset (see https://www.tensorflow.org/api_docs/python/tf/data/Dataset).

In [ ]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

We load the pretrained model and extend it using a Sequential model to classify the input into two classes (positive vs. negative sentiment).
Note that we set the model as `trainable` which means all parameters can be retrained. Since those parameters already have some weights and biases that work well, it is likely that the final weights and biases are close to what we already have in this part of the model.

In [ ]:
hub_layer = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim50/2",
                           input_shape=[], dtype=tf.string, trainable=True)

model = tf.keras.models.Sequential([
    hub_layer,
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

Since the dataset we are using is large, we can choose the batch size we want to use to feed the data into our model. Before batching the data, we can shuffle it. To have the same shuffling outcome everytime we run this code on our machine, we can set a seed for the random shuffle. Here we use `10000`. We can use any number we want.

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 5s 104ms/step - loss: 0.6807 - accuracy: 0.5662 - val_loss: 0.6037 - val_accuracy: 0.7066
Epoch 2/10
30/30 [==============================] - 3s 92ms/step - loss: 0.5607 - accuracy: 0.7597 - val_loss: 0.4973 - val_accuracy: 0.7905
Epoch 3/10
30/30 [==============================] - 3s 92ms/step - loss: 0.4275 - accuracy: 0.8517 - val_loss: 0.3926 - val_accuracy: 0.8448
Epoch 4/10
30/30 [==============================] - 3s 92ms/step - loss: 0.3018 - accuracy: 0.8974 - val_loss: 0.3385 - val_accuracy: 0.8595
Epoch 5/10
30/30 [==============================] - 3s 95ms/step - loss: 0.2172 - accuracy: 0.9326 - val_loss: 0.3145 - val_accuracy: 0.8690
Epoch 6/10
30/30 [==============================] - 3s 94ms/step - loss: 0.1586 - accuracy: 0.9538 - val_loss: 0.3067 - val_accuracy: 0.8735
Epoch 7/10
30/30 [==============================] - 3s 93ms/step - loss: 0.1161 - accuracy: 0.9714 - val_loss: 0.3072 - val_accuracy: 0.8725
Epoch 8/10
3

In [ ]:
model.evaluate(test_data.batch(512), verbose=2)

49/49 - 2s - loss: 0.3633 - accuracy: 0.8579


[0.36330464482307434, 0.8579199910163879]

In [ ]:
new_reviews = ["This was a great movie", "This was a very bad movie"]

In [ ]:
new_reviews_array = np.array(new_reviews)

In [ ]:
model(new_reviews_array)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.9793732 ],
       [0.11797823]], dtype=float32)>